In [1]:
from keras.preprocessing import sequence
import pandas as pd
import numpy as np

/home/huypham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('train_encode.csv')

In [3]:
df.head()

,descVec,wordIdVec
0,"[2427, 3953, 4687, 2416, 2337, 772, 40, 4878, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[2427, 3209, 3954, 4687, 0, 3730, 4945, 40, 12...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[2427, 3209, 3954, 4687, 2195, 2433, 3324, 259...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[2427, 5288, 4809, 4093, 5335, 3253, 872, 3324...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[2799, 2031, 380, 3412, 772, 183, 40, 2061, 24...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
max([len(i) for i in df.descVec])

270

In [5]:
maxlen = 500
max_features = 6000

In [11]:
input_train = [list(map(int, i.strip('[').strip(']').split(','))) for i in df.descVec]
y_train = [list(map(int, i[1:-1].replace(', ', ' ').split(' '))) for i in df.wordIdVec]
input_train_pad = sequence.pad_sequences(input_train, maxlen=maxlen, padding='post')
y_train_pad = sequence.pad_sequences(y_train, maxlen=maxlen, padding='post')

from keras.utils import to_categorical
y_train_cat = [to_categorical(i, num_classes=2) for i in y_train_pad]

In [7]:
testdf = pd.read_csv('test_encode.csv')

input_test = [list(map(int, i.strip('[').strip(']').split(','))) for i in testdf.descVec]
y_test = [list(map(float, i[1:-1].replace(', ', ' ').split(' '))) for i in testdf.wordIdVec]
input_test_pad = sequence.pad_sequences(input_test, maxlen=maxlen, padding='post')
y_test_pad = sequence.pad_sequences(y_test, maxlen=maxlen, padding='post')

In [8]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Embedding, Dropout, Activation, TimeDistributed, Bidirectional

In [9]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=50, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(2, activation="softmax")))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 50)           300000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500, 200)          120800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 2)            402       
Total params: 421,202
Trainable params: 421,202
Non-trainable params: 0
_________________________________________________________________


In [12]:
res = model.fit(input_train_pad, np.array(y_train_cat), batch_size=32, epochs=1, validation_split=0.1)

Train on 8684 samples, validate on 965 samples
Epoch 1/1
8684/8684 [==============================] - 279s 32ms/step - loss: 0.0116 - acc: 0.9952 - val_loss: 9.7526e-04 - val_acc: 0.9997


# TEST

In [13]:
# Original data
i = 111
print(input_test_pad[i])
print(y_test_pad[i])

[2427 3209 3954 4687 3210 4726 2711 4760 5288 4809 2482 1659   40 1200
 1659 4196 1408 1556 3135 3602  191    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [14]:
# Model prediction
p = model.predict(np.array([input_test_pad[i]]))
p = np.argmax(p, axis=-1)
print(p[0])

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 